# SOUND LOCALIZATION OF WILDLIFE

An argument based on sound localization in the wild can be solved using an implementation of the time difference of arrival (TDOA) algorithm in Python. TDOA is a technique used to determine the position of a sound source by measuring the time delay between the arrival of the sound at two or more microphones.



In [1]:
import numpy as np

def tdoa_localization(mic1, mic2, mic3, sound_speed):
    t1 = mic1[0]
    t2 = mic2[0]
    t3 = mic3[0]
    
    x1 = mic1[1]
    y1 = mic1[2]
    
    x2 = mic2[1]
    y2 = mic2[2]
    
    x3 = mic3[1]
    y3 = mic3[2]
    
    A = np.array([[x2-x1, y2-y1], [x3-x1, y3-y1]])
    b = np.array([(t2-t1)*sound_speed, (t3-t1)*sound_speed])
    x, y = np.linalg.solve(A, b)
    return x + x1, y + y1

# Example microphone locations and time differences
mic1 = (0, 0, 0)
mic2 = (0.1, 1, 0)
mic3 = (0.05, 0, 1)
sound_speed = 340 # m/s

source_position = tdoa_localization(mic1, mic2, mic3, sound_speed)
print("Source position:", source_position)


Source position: (34.0, 17.0)


In [2]:
import pandas as pd

# CREATING DATASET

In [9]:
#Creating Data
data = {
    "Features":['Microphone_array','Sound_card','Amplifier','ADC','Cover_area_sqm','Animal_type_avg_threatlvl'],
    "Locations":['Jim_Corbett_National_Park','Gir_Forest_National_Park','Karizanga_National_Park','Kanha_National_Park','Ranthambore_National_Park','Bandipur_National_Park'],
    #"Estimated_values_or_frequencies":[30000,18,17000,4096,1318.54,7]
    "Throughput_value":[36000,31000,28000,34000,28000,39000],
    "Sound_card_opt":[18,17,19,12,20,11],
    "Amplifier":[17000,19000,15000,160000,14000,15000],
    "ADC_Volts":[3569,4012,3894,3587,4078,3500],
    "Cover_area_sqm":[1318.54,1410.3,1537.53,940,1334,868.63],
    "Threat_level":[7,8.5,8,6.5,9.5,6]   
}
wild = pd.DataFrame(data)
wild


Features                  Locations  Throughput_value  \
0           Microphone_array  Jim_Corbett_National_Park             36000   
1                 Sound_card   Gir_Forest_National_Park             31000   
2                  Amplifier    Karizanga_National_Park             28000   
3                        ADC        Kanha_National_Park             34000   
4             Cover_area_sqm  Ranthambore_National_Park             28000   
5  Animal_type_avg_threatlvl     Bandipur_National_Park             39000   

   Sound_card_opt  Amplifier  ADC_Volts  Cover_area_sqm  Threat_level  
0              18      17000       3569         1318.54           7.0  
1              17      19000       4012         1410.30           8.5  
2              19      15000       3894         1537.53           8.0  
3              12     160000       3587          940.00           6.5  
4              20      14000       4078         1334.00           9.5  
5              11      15000       3500          868.63           6.0

In [10]:
wild = wild.set_index("Features")

In [11]:
wild.head()

Locations  Throughput_value  Sound_card_opt  \
Features                                                                        
Microphone_array  Jim_Corbett_National_Park             36000              18   
Sound_card         Gir_Forest_National_Park             31000              17   
Amplifier           Karizanga_National_Park             28000              19   
ADC                     Kanha_National_Park             34000              12   
Cover_area_sqm    Ranthambore_National_Park             28000              20   

                  Amplifier  ADC_Volts  Cover_area_sqm  Threat_level  
Features                                                              
Microphone_array      17000       3569         1318.54           7.0  
Sound_card            19000       4012         1410.30           8.5  
Amplifier             15000       3894         1537.53           8.0  
ADC                  160000       3587          940.00           6.5  
Cover_area_sqm        14000       4078         1334.00           9.5

In [12]:
wild.to_csv("features_and_locations.csv")

In [13]:
#Label Encoding
from sklearn import preprocessing
LE = preprocessing.LabelEncoder()
wild['Locations'] = LE.fit_transform(wild['Locations'])
wild["Locations"].unique()

array([2, 1, 4, 3, 5, 0])

# A * algorithm

In [17]:
import heapq

class Microphone:
    def __init__(self, id, frequency, location):
        self.id = id
        self.frequency = frequency
        self.location = location

class SoundSource:
    def __init__(self, location):
        self.location = location

class Edge:
    def __init__(self, source, target, weight):
        self.source = source
        self.target = target
        self.weight = weight

def heuristic(microphone, sound_source):
    # Estimate the cost from the microphone to the sound source
    # using the Euclidean distance
    x1, y1 = microphone.location
    x2, y2 = sound_source.location
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def a_star(microphones, sound_source):
    start = microphones[0]
    goal = microphones[0]
    heap = [(0 + heuristic(start, sound_source), 0, start.id)]
    visited = set()
    while heap:
        (f, g, current_id) = heapq.heappop(heap)
        if current_id in visited:
            continue
        visited.add(current_id)
        current = [microphone for microphone in microphones if microphone.id == current_id][0]
        if current.frequency > goal.frequency:
            goal = current
        for microphone in microphones:
            if microphone.id == current.id:
                continue
            cost = g + heuristic(current, microphone)
            heapq.heappush(heap, (cost + heuristic(microphone, sound_source), cost, microphone.id))
    return goal

# Example usage
microphones = [
    Microphone(0, 10, (0, 0)),
    Microphone(1, 20, (0, 1)),
    Microphone(2, 30, (1, 0)),
    Microphone(3, 40, (1, 1))
]
sound_source = SoundSource((0.5, 0.5))
best_microphone = a_star(microphones, sound_source)
print("Best microphone:", best_microphone.id)


Best microphone: 3


# HILL CLIMBING ALGORITHM

In [18]:
import numpy as np
import random

def sound_localization(initial_location, max_distance, steps):
    current_location = initial_location
    best_location = current_location
    best_score = get_score(current_location)

    for i in range(steps):
        # Generate a random move
        move = np.random.uniform(-max_distance, max_distance, 2)
        new_location = current_location + move
        new_score = get_score(new_location)

        # If the new location has a better score, update the current location and best location
        if new_score > best_score:
            current_location = new_location
            best_location = current_location
            best_score = new_score
    
    return best_location

def get_score(location):
    # This function calculates a score for a given location based on the strength of the sound signal at that location
    # In a real-world scenario, this would involve measuring the sound level at the microphone at the given location
    # For this example, we'll just return a random score
    return random.uniform(0, 100)

# Example usage
initial_location = np.array([0, 0])
max_distance = 1
steps = 100
best_location = sound_localization(initial_location, max_distance, steps)
print("Best location:", best_location)


Best location: [0.45258118 0.59491848]


# PLOTLY VISUALIZATION

In [16]:
import pandas as pd
import plotly.express as px

# Load the features and locations data
#wd1 = pd.read_csv("features_and_locations.csv")
#wd1.set_index("Features")
# Split the data into features and target (location)
X = wild.drop("Locations", axis=1)
y = wild["Locations"]

# wd1['Features']= LE.fit_transform(wd['Features'])
# wd1["Features"].unique()
# Train the random forest model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, y)

# Predict the locations using the trained model
y_pred = model.predict(X)

# Add the predicted locations to the data frame
wild["Threat_level"] = y_pred

# Create a scatter plot of the actual vs predicted locations
fig = px.scatter(wild, x="ADC_Volts", y="Threat_level",color='Cover_area_sqm',
                 title="Amplifier voltage based on park",
                 labels={"ADC_Volts": "Watt(h)Volts",
                         "Threat_level": "Specatator_animal_nonaffinity"})

# Show the plot
fig.show()
